In [ ]:
import scrape

In [ ]:
d = scrape.scrape()

In [ ]:
d['SpiderID'].__len__()

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame.from_dict(d, orient='index').T

In [ ]:
df.to_csv('resume_data.csv')

In [ ]:
d.keys()

In [ ]:
df.info()

In [ ]:
df.nunique()

In [ ]:
df.shape

In [ ]:
df.drop

In [ ]:
df1 = pd.read_csv('resume_data.csv')

In [ ]:
df1['Industry'].nunique()